In [1]:
import findspark
findspark.init('C:\Spark\spark-3.0.1-bin-hadoop2.7')
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('recommender').getOrCreate()

In [2]:
data = spark.read.csv('movielens_ratings.csv',
                      inferSchema=True,
                      header=True)

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data.show(5)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



In [5]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [6]:
data.groupBy('userID').count().orderBy('userID').show(5)

+------+-----+
|userID|count|
+------+-----+
|     0|   49|
|     1|   49|
|     2|   46|
|     3|   48|
|     4|   55|
+------+-----+
only showing top 5 rows



In [7]:
# Smaller dataset so we will use 0.8 / 0.2
training, test = data.randomSplit([0.8, 0.2])

In [8]:
als = ALS(maxIter=5, regParam=0.01, 
          userCol="userId", 
          itemCol="movieId", 
          ratingCol="rating")

model = als.fit(training)

In [9]:
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    26|-2.1512482|
|     31|   1.0|    27| 1.7858771|
|     31|   1.0|    19|0.18949133|
|     31|   1.0|    29| 2.0617073|
|     85|   1.0|    12|0.41962832|
|     85|   1.0|     4|  2.090305|
|     85|   4.0|     7| 3.0730243|
|     85|   1.0|    29| 1.1852045|
|     65|   1.0|    16| 0.5115368|
|     65|   2.0|     3| 1.5624579|
|     65|   2.0|    15| 1.1699452|
|     53|   3.0|    13| 4.2538056|
|     53|   1.0|    23| 1.1034933|
|     53|   1.0|     7| 2.3743136|
|     53|   3.0|    14| 5.2616744|
|     78|   1.0|    28| 1.2707193|
|     78|   1.0|    22| 1.3429934|
|     78|   1.0|    13| 1.0663147|
|     34|   4.0|     2|  3.803225|
|     81|   5.0|    28|-0.5304088|
+-------+------+------+----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol='rating',
                                predictionCol='prediction')

In [12]:
rmse = evaluator.evaluate(predictions)

In [13]:
print('RMSE')
print(rmse)

RMSE
1.532347176170291


In [14]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])

In [15]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     10|    11|
|     32|    11|
|     43|    11|
|     45|    11|
|     47|    11|
|     48|    11|
|     61|    11|
|     77|    11|
|     89|    11|
|     90|    11|
+-------+------+



In [16]:
recommendations = model.transform(single_user)

In [17]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     89|    11| 2.7203965|
|     32|    11| 2.4148073|
|     61|    11| 2.2886453|
|      9|    11|  2.141322|
|     48|    11| 0.8344513|
|     43|    11|  0.815569|
|     45|    11|0.65236646|
|     10|    11| 0.5439737|
|     90|    11|-0.3386134|
|     47|    11|-1.3321455|
|     77|    11|-2.0953267|
+-------+------+----------+

